In [1]:
import requests
from bs4 import *
import json
import random

In [2]:
def randomId():
    return str(int(random.random()*1000000)+int(random.random()*1000))


In [3]:
def detailed(link):
    content=requests.get("https://www.espncricinfo.com"+link).content
    soup=BeautifulSoup(content,"html")
    results = soup.find_all("div", class_="ds-rounded-lg ds-mt-2")
    scorecard=[]
    for i in results:
        team={}
        team["id"]="t"+randomId()
        team["team"] = i.find("span", class_="ds-text-title-xs ds-font-bold ds-capitalize").text
        batting = i.find("table", class_="ci-scorecard-table").find("tbody")
        b_scores=[]
        scores = batting.find_all("tr")
        for p in scores:
            ps={}
            ps["id"] = "ba"+randomId()
            s=list(map(lambda x:x.text,p.find_all("td")))
            if "extras" in s[0].lower():
                break
            if s[0]!="":
                ps["player"]=s[0].replace('''\xa0''',"")
                ps["status"] = s[1]
                ps["runs"] = s[2]
                ps["balls"] = s[3]
                ps["fours"] = s[5]
                ps["sixes"] = s[6]
                ps["sr"] = s[7]
                b_scores+=[ps]
        bowling=i.find_all("table")[1].find("tbody")
        bo_scores=[]
        scores = bowling.find_all("tr")
        for p in scores:
            ps = {}
            ps["id"] = "bo"+randomId()
            s = list(map(lambda x: x.text, p.find_all("td")))
            if len(s)==11:
                ps["player"] = s[0].replace('''\xa0''', "")
                ps["overs"] = s[1]
                ps["maiden"] = s[2]
                ps["runs"] = s[3]
                ps["wickets"] = s[4]
                ps["econ"] = s[5]
                ps["wides"] = s[9]
                ps["noBalls"] = s[10]
                bo_scores += [ps]
        team["batting"] = b_scores
        team["bowling"] = bo_scores
        scorecard+=[team]
    return scorecard


In [4]:
res = requests.get(
    "https://www.espncricinfo.com/series/icc-men-s-t20-world-cup-2022-23-1298134/match-schedule-fixtures-and-results")
content = res.content
soup = BeautifulSoup(content, "html")
data=[]
try:
    d_temp=""
    c=1
    for i in soup.find_all("div", class_="ds-p-4"):
        date = i.find("div", class_="ds-text-compact-xs ds-font-bold ds-w-24")
        link=i.find("a")
        teams = i.find_all("div", class_="ci-team-score")
        m={}
        m["id"]="m"+str(123456*c)
        c+=1
        if None not in [date,link,teams]:
            t1,t2=teams
            if date.text.replace(" ","")=="":
                m["date"]=d_temp
            else:
                m["date"]=date.text
                d_temp=date.text
            t1_name = t1.find("p")
            t2_name = t2.find("p")
            t1_score = t1.find("strong")
            t2_score = t2.find("strong")
            result = i.find("p", class_="ds-text-tight-s")
            if None not in [t1_name,t2_name,t1_score,t2_score]:
                m["t1Name"] = t1_name.text
                m["t2Name"] = t2_name.text
                m["t1Score"] = t1_score.text
                m["t2Score"] = t2_score.text
                m["result"]=result.text
                m["scorecard"] = detailed(link.get("href"))
                data+=[m]
except Exception:
    print(Exception)


In [5]:
data

[{'id': 'm123456',
  'date': 'Sun, 16 Oct',
  't1Name': 'Namibia',
  't2Name': 'Sri Lanka',
  't1Score': '163/7',
  't2Score': '108',
  'result': 'Namibia won by 55 runs',
  'scorecard': [{'id': 't864135',
    'team': 'Namibia',
    'batting': [{'id': 'ba818318',
      'player': 'Michael van Lingen',
      'status': 'c Pramod Madushan b Chameera',
      'runs': '3',
      'balls': '6',
      'fours': '0',
      'sixes': '0',
      'sr': '50.00'},
     {'id': 'ba49443',
      'player': 'Divan la Cock',
      'status': 'c Shanaka b Pramod Madushan',
      'runs': '9',
      'balls': '9',
      'fours': '1',
      'sixes': '0',
      'sr': '100.00'},
     {'id': 'ba806522',
      'player': 'Jan Nicol Loftie-Eaton',
      'status': 'c †Mendis b Karunaratne',
      'runs': '20',
      'balls': '12',
      'fours': '1',
      'sixes': '2',
      'sr': '166.66'},
     {'id': 'ba293114',
      'player': 'Stephan Baard',
      'status': 'c DM de Silva b Pramod Madushan',
      'runs': '26',
   

In [6]:
with open("matches1.json","w") as fp:
    json.dump(data,fp,indent=5)